In [55]:
!pip install mediapipe==0.9.0.1 opencv-python==4.7.0.68 pandas scikit-learn==1.0.2 pyttsx3

In [1]:
import mediapipe as mp # Import mediapipe
import cv2 # Import opencv

In [2]:
mp_drawing = mp.solutions.drawing_utils # Drawing helpers
mp_holistic = mp.solutions.holistic # Mediapipe Solutions

# Live View

In [4]:
cap = cv2.VideoCapture(1)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
                        
        cv2.imshow('Live Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

# Capturing Landmarks & Exporting to CSV

In [5]:
import csv
import os
import numpy as np

In [6]:
num_coords = len(results.pose_landmarks.landmark)+len(results.face_landmarks.landmark)
num_coords

501

In [31]:
landmarks = ['class']
for val in range(1, num_coords+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

In [32]:
with open('coords_class.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [33]:
class_name = "Normal"

In [35]:
cap = cv2.VideoCapture(1)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        # Export coordinates
        try:
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            # Extract Face landmarks
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
            
            # Concate rows
            row = pose_row+face_row
            
            # Append class name 
            row.insert(0, class_name)
            
            # Export to CSV
            with open('coords_class.csv', mode='a', newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row) 
            
        except:
            pass
                        
        cv2.imshow(str(class_name), image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

# Training a custom model

In [36]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [37]:
df = pd.read_csv('coords.csv')

In [38]:
df.head()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
0,Normal,0.511904,0.073090,-0.606731,0.999782,0.521700,0.049005,-0.584288,0.999432,0.527605,...,-0.000977,0,0.533466,0.041678,0.006960,0,0.535358,0.040076,0.007261,0
1,Normal,0.512641,0.073134,-0.613237,0.999799,0.522867,0.048641,-0.591737,0.999475,0.528902,...,-0.001108,0,0.534035,0.040976,0.005380,0,0.535792,0.039702,0.005508,0
2,Normal,0.514952,0.073192,-0.613761,0.999816,0.525230,0.048578,-0.591797,0.999520,0.531351,...,-0.000873,0,0.532899,0.041810,0.005634,0,0.534649,0.040645,0.005842,0
3,Normal,0.515097,0.073145,-0.613461,0.999832,0.525029,0.048562,-0.591436,0.999563,0.531026,...,-0.000840,0,0.529658,0.039693,0.006064,0,0.531384,0.038409,0.006264,0
4,Normal,0.512284,0.070971,-0.601240,0.999846,0.522377,0.046916,-0.577947,0.999599,0.527943,...,-0.000625,0,0.523451,0.038944,0.006190,0,0.525196,0.037469,0.006425,0


In [39]:
df.tail()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
1486,Leg Pain,0.435291,0.126975,-0.536476,0.999938,0.442637,0.104553,-0.526022,0.999866,0.448308,...,-0.001913,0,0.450727,0.103315,-0.002358,0,0.451854,0.100786,-0.002654,0
1487,Leg Pain,0.435875,0.128234,-0.506256,0.999941,0.443073,0.105654,-0.496442,0.999875,0.448636,...,-0.002779,0,0.450579,0.103646,-0.003902,0,0.451716,0.101213,-0.004318,0
1488,Leg Pain,0.436551,0.130644,-0.539978,0.999946,0.443941,0.107769,-0.528946,0.999885,0.449563,...,-0.002276,0,0.452970,0.107961,-0.002562,0,0.453951,0.105224,-0.002815,0
1489,Leg Pain,0.437960,0.132852,-0.506143,0.999950,0.445299,0.110138,-0.496337,0.999894,0.450955,...,-0.002892,0,0.453134,0.109491,-0.003950,0,0.454263,0.107027,-0.004303,0
1490,Leg Pain,0.441210,0.134195,-0.495235,0.999953,0.448147,0.111852,-0.484749,0.999901,0.453703,...,-0.002666,0,0.459867,0.112227,-0.003914,0,0.461162,0.109528,-0.004297,0


In [40]:
df[df['class']=='Normal']

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
0,Normal,0.511904,0.073090,-0.606731,0.999782,0.521700,0.049005,-0.584288,0.999432,0.527605,...,-0.000977,0,0.533466,0.041678,0.006960,0,0.535358,0.040076,0.007261,0
1,Normal,0.512641,0.073134,-0.613237,0.999799,0.522867,0.048641,-0.591737,0.999475,0.528902,...,-0.001108,0,0.534035,0.040976,0.005380,0,0.535792,0.039702,0.005508,0
2,Normal,0.514952,0.073192,-0.613761,0.999816,0.525230,0.048578,-0.591797,0.999520,0.531351,...,-0.000873,0,0.532899,0.041810,0.005634,0,0.534649,0.040645,0.005842,0
3,Normal,0.515097,0.073145,-0.613461,0.999832,0.525029,0.048562,-0.591436,0.999563,0.531026,...,-0.000840,0,0.529658,0.039693,0.006064,0,0.531384,0.038409,0.006264,0
4,Normal,0.512284,0.070971,-0.601240,0.999846,0.522377,0.046916,-0.577947,0.999599,0.527943,...,-0.000625,0,0.523451,0.038944,0.006190,0,0.525196,0.037469,0.006425,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
587,Normal,0.479228,0.080622,-0.452016,0.999973,0.486364,0.062823,-0.442870,0.999891,0.491565,...,-0.005104,0,0.492802,0.062276,-0.001793,0,0.494129,0.061365,-0.001881,0
588,Normal,0.480503,0.080630,-0.456100,0.999974,0.487549,0.062703,-0.446116,0.999894,0.492542,...,-0.005955,0,0.493732,0.060789,-0.003181,0,0.494900,0.059762,-0.003308,0
589,Normal,0.481294,0.080493,-0.461186,0.999974,0.488212,0.062488,-0.450937,0.999895,0.493098,...,-0.004627,0,0.493807,0.061112,-0.001442,0,0.494928,0.060318,-0.001552,0
590,Normal,0.481630,0.080481,-0.459817,0.999974,0.488592,0.062433,-0.449864,0.999896,0.493488,...,-0.005387,0,0.494900,0.060228,-0.002302,0,0.496091,0.059168,-0.002396,0


In [41]:
df[df['class']=='Heart Pain']

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
592,Heart Pain,0.496618,0.420941,-0.722692,0.996428,0.505470,0.382103,-0.689742,0.996299,0.511422,...,-0.001211,0,0.496300,0.357862,0.001406,0,0.499322,0.354421,0.001174,0
593,Heart Pain,0.481142,0.399718,-0.841168,0.996784,0.496949,0.366887,-0.803666,0.996667,0.505541,...,0.002035,0,0.492886,0.353794,0.005499,0,0.495547,0.350320,0.005497,0
594,Heart Pain,0.471625,0.393286,-0.793635,0.997104,0.489609,0.362569,-0.759230,0.996994,0.500124,...,0.003581,0,0.491012,0.353152,0.006954,0,0.493684,0.349402,0.007012,0
595,Heart Pain,0.468130,0.389107,-0.777927,0.997391,0.485754,0.359413,-0.744241,0.997290,0.496934,...,0.002417,0,0.485439,0.352998,0.004492,0,0.488014,0.349554,0.004433,0
596,Heart Pain,0.466213,0.386443,-0.772679,0.997645,0.483005,0.358614,-0.738973,0.997547,0.494347,...,0.004725,0,0.485468,0.352612,0.007475,0,0.488167,0.348643,0.007575,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
878,Heart Pain,0.555295,0.126917,-0.365423,0.999988,0.567360,0.105391,-0.352056,0.999959,0.574010,...,-0.005072,0,0.575397,0.105605,-0.009339,0,0.577256,0.103662,-0.010056,0
879,Heart Pain,0.557499,0.126829,-0.367667,0.999988,0.570240,0.105145,-0.353206,0.999960,0.577230,...,-0.005043,0,0.579295,0.104909,-0.008562,0,0.581131,0.102902,-0.009224,0
880,Heart Pain,0.557676,0.126405,-0.370475,0.999989,0.570263,0.104819,-0.357168,0.999961,0.577201,...,-0.005375,0,0.578072,0.102806,-0.008440,0,0.579827,0.101001,-0.009082,0
881,Heart Pain,0.558746,0.126208,-0.388741,0.999989,0.571312,0.104795,-0.375740,0.999962,0.578219,...,-0.004631,0,0.577486,0.100726,-0.008011,0,0.579216,0.098918,-0.008681,0


In [42]:
df[df['class']=='Head Ache']

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
883,Head Ache,0.459665,0.471031,-0.757312,0.999701,0.478707,0.433076,-0.744288,0.999529,0.488046,...,-0.015365,0,0.494827,0.435235,-0.007116,0,0.497664,0.431712,-0.007375,0
884,Head Ache,0.459995,0.478169,-0.586908,0.999724,0.478138,0.438559,-0.571159,0.999503,0.487477,...,-0.013913,0,0.494071,0.436978,-0.005637,0,0.496680,0.434287,-0.005939,0
885,Head Ache,0.457902,0.478193,-0.667104,0.999738,0.476180,0.439308,-0.655199,0.999483,0.485989,...,-0.013726,0,0.495269,0.434700,-0.005254,0,0.497995,0.432170,-0.005604,0
886,Head Ache,0.456207,0.478264,-0.671947,0.999756,0.474404,0.440079,-0.665556,0.999494,0.484373,...,-0.015397,0,0.496004,0.432404,-0.007463,0,0.498599,0.429696,-0.007872,0
887,Head Ache,0.456904,0.477430,-0.561775,0.999777,0.474808,0.439376,-0.548464,0.999525,0.484640,...,-0.015204,0,0.496120,0.433025,-0.006958,0,0.498672,0.430310,-0.007318,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1183,Head Ache,0.495438,0.123790,-0.268188,0.999895,0.501966,0.102960,-0.268986,0.999617,0.506689,...,0.002347,0,0.521304,0.081533,0.006389,0,0.522356,0.078174,0.006607,0
1184,Head Ache,0.495679,0.123920,-0.253728,0.999898,0.502272,0.103133,-0.255094,0.999630,0.507026,...,0.002742,0,0.527437,0.082323,0.008131,0,0.528893,0.079387,0.008351,0
1185,Head Ache,0.495025,0.121626,-0.263926,0.999899,0.501709,0.100665,-0.264848,0.999634,0.506485,...,-0.007324,0,0.510259,0.108987,-0.011630,0,0.510815,0.105629,-0.012238,0
1186,Head Ache,0.494146,0.123191,-0.263456,0.999898,0.500589,0.102202,-0.265099,0.999631,0.505169,...,-0.009040,0,0.510439,0.113892,-0.014753,0,0.511117,0.110075,-0.015558,0


In [43]:
df[df['class']=='Leg Pain']

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
1188,Leg Pain,0.506805,0.158293,-0.438630,0.998710,0.512937,0.135490,-0.421441,0.997036,0.517709,...,-0.002812,0,0.516429,0.140211,0.001916,0,0.518443,0.137045,0.001983,0
1189,Leg Pain,0.502840,0.173617,-0.575354,0.998837,0.509649,0.147391,-0.568145,0.997324,0.515093,...,-0.001214,0,0.512577,0.151547,0.002868,0,0.514182,0.149754,0.002904,0
1190,Leg Pain,0.493936,0.214218,-0.609037,0.998952,0.497738,0.184900,-0.606587,0.997587,0.503561,...,-0.004862,0,0.502536,0.185225,-0.001407,0,0.503976,0.183010,-0.001539,0
1191,Leg Pain,0.479900,0.244047,-0.673495,0.999052,0.483165,0.216229,-0.675271,0.997810,0.489115,...,-0.004826,0,0.492986,0.192366,-0.001012,0,0.494170,0.189081,-0.001059,0
1192,Leg Pain,0.465626,0.269685,-0.650111,0.999141,0.469685,0.241497,-0.655386,0.998009,0.475579,...,-0.006772,0,0.483918,0.230926,-0.004438,0,0.485162,0.228358,-0.004699,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1486,Leg Pain,0.435291,0.126975,-0.536476,0.999938,0.442637,0.104553,-0.526022,0.999866,0.448308,...,-0.001913,0,0.450727,0.103315,-0.002358,0,0.451854,0.100786,-0.002654,0
1487,Leg Pain,0.435875,0.128234,-0.506256,0.999941,0.443073,0.105654,-0.496442,0.999875,0.448636,...,-0.002779,0,0.450579,0.103646,-0.003902,0,0.451716,0.101213,-0.004318,0
1488,Leg Pain,0.436551,0.130644,-0.539978,0.999946,0.443941,0.107769,-0.528946,0.999885,0.449563,...,-0.002276,0,0.452970,0.107961,-0.002562,0,0.453951,0.105224,-0.002815,0
1489,Leg Pain,0.437960,0.132852,-0.506143,0.999950,0.445299,0.110138,-0.496337,0.999894,0.450955,...,-0.002892,0,0.453134,0.109491,-0.003950,0,0.454263,0.107027,-0.004303,0


In [44]:
X = df.drop('class', axis=1) # features
y = df['class'] # target value

In [45]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)

In [46]:
y_test

466         Normal
631     Heart Pain
1443      Leg Pain
1172     Head Ache
458         Normal
           ...    
681     Heart Pain
1262      Leg Pain
83          Normal
413         Normal
62          Normal
Name: class, Length: 448, dtype: object

## Classification Model

In [47]:
from sklearn.pipeline import make_pipeline 
from sklearn.preprocessing import StandardScaler 

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [48]:
pipelines = {
    'lr':make_pipeline(StandardScaler(), LogisticRegression(max_iter=10000)),
    'rc':make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf':make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}

In [49]:
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train, y_train)
    fit_models[algo] = model

In [50]:
fit_models

{'lr': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('logisticregression', LogisticRegression(max_iter=10000))]),
 'rc': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('ridgeclassifier', RidgeClassifier())]),
 'rf': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('randomforestclassifier', RandomForestClassifier())]),
 'gb': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('gradientboostingclassifier', GradientBoostingClassifier())])}

In [51]:
fit_models['rc'].predict(X_test)

array(['Normal', 'Heart Pain', 'Leg Pain', 'Head Ache', 'Normal',
       'Heart Pain', 'Leg Pain', 'Leg Pain', 'Normal', 'Normal',
       'Leg Pain', 'Leg Pain', 'Leg Pain', 'Head Ache', 'Normal',
       'Head Ache', 'Leg Pain', 'Normal', 'Normal', 'Heart Pain',
       'Heart Pain', 'Head Ache', 'Normal', 'Normal', 'Head Ache',
       'Heart Pain', 'Leg Pain', 'Normal', 'Leg Pain', 'Leg Pain',
       'Normal', 'Normal', 'Leg Pain', 'Leg Pain', 'Head Ache',
       'Leg Pain', 'Head Ache', 'Normal', 'Leg Pain', 'Normal',
       'Head Ache', 'Heart Pain', 'Normal', 'Leg Pain', 'Head Ache',
       'Normal', 'Normal', 'Normal', 'Head Ache', 'Leg Pain',
       'Heart Pain', 'Normal', 'Normal', 'Normal', 'Heart Pain',
       'Leg Pain', 'Normal', 'Leg Pain', 'Heart Pain', 'Normal', 'Normal',
       'Normal', 'Leg Pain', 'Heart Pain', 'Leg Pain', 'Heart Pain',
       'Normal', 'Head Ache', 'Head Ache', 'Normal', 'Head Ache',
       'Head Ache', 'Head Ache', 'Heart Pain', 'Normal', 'Normal',
  

## Dump model

In [52]:
from sklearn.metrics import accuracy_score # Accuracy metrics 
import pickle 

In [53]:
for algo, model in fit_models.items():
    yhat = model.predict(X_test)
    print(algo, accuracy_score(y_test, yhat))

lr 1.0
rc 0.9977678571428571
rf 0.9955357142857143
gb 0.9977678571428571


In [54]:
fit_models['rf'].predict(X_test)

array(['Normal', 'Heart Pain', 'Leg Pain', 'Head Ache', 'Normal',
       'Heart Pain', 'Leg Pain', 'Leg Pain', 'Normal', 'Normal',
       'Leg Pain', 'Leg Pain', 'Leg Pain', 'Head Ache', 'Normal',
       'Head Ache', 'Leg Pain', 'Normal', 'Normal', 'Heart Pain',
       'Heart Pain', 'Head Ache', 'Normal', 'Normal', 'Head Ache',
       'Heart Pain', 'Leg Pain', 'Normal', 'Leg Pain', 'Leg Pain',
       'Normal', 'Normal', 'Leg Pain', 'Leg Pain', 'Head Ache',
       'Leg Pain', 'Head Ache', 'Normal', 'Leg Pain', 'Normal',
       'Head Ache', 'Normal', 'Normal', 'Leg Pain', 'Head Ache', 'Normal',
       'Normal', 'Normal', 'Head Ache', 'Leg Pain', 'Heart Pain',
       'Normal', 'Normal', 'Normal', 'Heart Pain', 'Leg Pain', 'Normal',
       'Leg Pain', 'Heart Pain', 'Normal', 'Normal', 'Normal', 'Leg Pain',
       'Heart Pain', 'Leg Pain', 'Heart Pain', 'Normal', 'Head Ache',
       'Head Ache', 'Normal', 'Head Ache', 'Head Ache', 'Head Ache',
       'Heart Pain', 'Heart Pain', 'Normal', 'H

In [55]:
y_test

466         Normal
631     Heart Pain
1443      Leg Pain
1172     Head Ache
458         Normal
           ...    
681     Heart Pain
1262      Leg Pain
83          Normal
413         Normal
62          Normal
Name: class, Length: 448, dtype: object

In [56]:
with open('body_language.pkl', 'wb') as f:
    pickle.dump(fit_models['rf'], f)

# Detection

In [57]:
import pyttsx3
with open('body_language.pkl', 'rb') as f:
    model = pickle.load(f)

In [71]:
model

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestclassifier', RandomForestClassifier())])

In [58]:
# feature names
fnames = []
for val in range(1, num_coords+1):
    fnames += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

In [59]:
engine = pyttsx3.init()
cap = cv2.VideoCapture(1)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        # Export coordinates
        try:
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            # Extract Face landmarks
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
            
            # Concate rows
            row = pose_row+face_row
            
            # Make Detections
            X = pd.DataFrame([row], columns=fnames)
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]
            #print(body_language_class, body_language_prob)
            
            # Grab ear coords
            coords = tuple(np.multiply(
                            np.array(
                                (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
                                 results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y))
                        , [640,480]).astype(int))
            
            cv2.rectangle(image, 
                          (coords[0], coords[1]+5), 
                          (coords[0]+len(body_language_class)*20, coords[1]-30), 
                          (245, 117, 16), -1)
            cv2.putText(image, body_language_class, coords, 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Get status box
            cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)
            
            # Display Class
            cv2.putText(image, 'CLASS'
                        , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, body_language_class
                        , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Display Probability
            cv2.putText(image, 'PROB'
                        , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2))
                        , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Audio Alert
            if body_language_class != "Normal":
                engine.say(str(body_language_class)+"detected")
                engine.runAndWait()
            
        except:
            pass
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()